## **データ整理**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
from tqdm import tqdm

from myUtils import reduce_mem_usage

import gc

#### [参考]submissionファイルに合わせてaggurigationした学習データ作成
https://www.kaggle.com/sachina/convert-accuracy-to-uncertainty-poisson

In [2]:
def createTrainSet(sales_train_s,train_sales, name, group_level, X = False):
    sales_total = CreateSales(train_sales,name, group_level)
    if(X == True):
        sales_total = sales_total.rename(index = lambda s:  s + '_X')
    sales_train_s = sales_train_s.append(sales_total)
    return(sales_train_s)

def get_agg_df(train_sales):
    total = ['Total']
    train_sales['Total'] = 'Total'
    train_sales['state_cat'] = train_sales.state_id + "_" + train_sales.cat_id
    train_sales['state_dept'] = train_sales.state_id + "_" + train_sales.dept_id
    train_sales['store_cat'] = train_sales.store_id + "_" + train_sales.cat_id
    train_sales['store_dept'] = train_sales.store_id + "_" + train_sales.dept_id
    train_sales['state_item'] = train_sales.state_id + "_" + train_sales.item_id
    train_sales['item_store'] = train_sales.item_id + "_" + train_sales.store_id
    total = ['Total']
    states = ['CA', 'TX', 'WI']
    num_stores = [('CA',4), ('TX',3), ('WI',3)]
    stores = [x[0] + "_" + str(y + 1) for x in num_stores for y in range(x[1])]
    cats = ['FOODS', 'HOBBIES', 'HOUSEHOLD']
    num_depts = [('FOODS',3), ('HOBBIES',2), ('HOUSEHOLD',2)]
    depts = [x[0] + "_" + str(y + 1) for x in num_depts for y in range(x[1])]
    state_cats = [state + "_" + cat for state in states for cat in cats]
    state_depts = [state + "_" + dept for state in states for dept in depts]
    store_cats = [store + "_" + cat for store in stores for cat in cats]
    store_depts = [store + "_" + dept for store in stores for dept in depts]
    prods = list(train_sales.item_id.unique())
    prod_state = [prod + "_" + state for prod in prods for state in states]
    prod_store = [prod + "_" + store for prod in prods for store in stores]
    cols = [i for i in train_sales.columns if i.startswith('F')]
    sales_train_s = train_sales[cols]
    sales_train_s = pd.DataFrame()
    sales_train_s = createTrainSet(sales_train_s,train_sales, total, 'Total', X=True) #1
    sales_train_s = createTrainSet(sales_train_s, train_sales,states, 'state_id', X=True) #2
    sales_train_s = createTrainSet(sales_train_s,train_sales, stores, 'store_id', X=True) #3
    sales_train_s = createTrainSet(sales_train_s,train_sales, cats, 'cat_id', X=True) #4
    sales_train_s = createTrainSet(sales_train_s,train_sales, depts, 'dept_id', X=True) #5
    sales_train_s = createTrainSet(sales_train_s,train_sales, state_cats, 'state_cat') #6
    sales_train_s = createTrainSet(sales_train_s,train_sales, state_depts, 'state_dept') #7
    sales_train_s = createTrainSet(sales_train_s,train_sales, store_cats, 'store_cat') #8
    sales_train_s = createTrainSet(sales_train_s,train_sales, store_depts, 'store_dept') #9
    sales_train_s = createTrainSet(sales_train_s,train_sales, prods, 'item_id', X=True) #10
    sales_train_s = createTrainSet(sales_train_s,train_sales, prod_state, 'state_item') #11
    sales_train_s = createTrainSet(sales_train_s,train_sales, prod_store, 'item_store')
    sales_train_s['id'] = sales_train_s.index
    return(sales_train_s)

In [4]:
sales_train_val = pd.read_csv(os.path.join("rawdata","sales_train_validation.csv")) # データの読み出し
sales_train_val = sales_train_val.set_index(["id","item_id","dept_id","cat_id","store_id","state_id"],drop=True)
display(sales_train_val)

,,,,,,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
id,item_id,dept_id,cat_id,store_id,state_id,,,,,,,,,,,,,,,,,,,,,
HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,0,3,1,4,1,0,...,2,0,0,0,0,0,1,0,0,1
FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,0,5,0,1,1,3,...,0,0,0,0,0,0,0,0,1,0
FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,2,4,1,8,5,2,...,2,1,0,2,0,1,0,0,1,0


In [5]:
get_agg_df(sales_train_val)

AttributeError: 'DataFrame' object has no attribute 'state_id'